# Introduction: Machine Learning for Geospatial Use Cases

In recent years, geospatial data has gained significant attention due to its diverse applications in fields such as urban planning, environmental monitoring, and disaster management. The ability to model and predict patterns using geospatial data is critical for informed decision-making. However, working with geospatial datasets presents unique challenges, including handling high-dimensional data, managing spatial dependencies, and integrating various data sources with different resolutions and formats.

This notebook focuses on leveraging **machine learning techniques**, particularly **XGBoost**, to address a geospatial use case. XGBoost is a powerful gradient boosting framework known for its efficiency and performance, especially in handling structured data. The workflow covered in this notebook includes the following stages:

1. **Data Extraction**: Collecting geospatial data from multiple sources, such as satellite imagery, sensor networks, and public databases.
2. **Data Preprocessing**: Cleaning and transforming the data to ensure compatibility with machine learning algorithms. This includes handling missing values, feature scaling, and encoding categorical variables.
3. **Feature Engineering**: Creating meaningful features from raw geospatial data, including distance calculations, spatial aggregations, and temporal features.
4. **Model Training and Evaluation**: Applying XGBoost to predict outcomes based on geospatial features. The model's performance is evaluated using appropriate metrics, and hyperparameters are tuned to improve accuracy.
5. **Visualization**: Using geospatial visualization tools to interpret model outputs and understand spatial patterns in the data.

The goal is to demonstrate how machine learning can be effectively applied to geospatial data, providing actionable insights and enhancing the decision-making process. This approach bridges the gap between traditional geospatial analysis and modern machine learning techniques, offering robust solutions to complex spatial problems.

**Note**: This notebook does not include the entirety of the work done to improve the quality of predictions. Additional efforts, such as further hyperparameter tuning, advanced feature engineering, and model ensembling, have been undertaken to enhance the model's performance.

## Importing Required Libraries

To build and evaluate the machine learning model for our geospatial use case, we start by importing the necessary libraries. These include both standard libraries and third-party tools for data manipulation, visualization, and machine learning.


In [ ]:
# Standard library imports
import os
import re
import time
from datetime import datetime, timezone
from urllib.parse import quote

# Third-party imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error, make_scorer
from xgboost import XGBRegressor
import optuna
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

In [ ]:
# Configuration and constants
BASE_URL = "https://metrics.run.lithops.cloud/api/v1"
AUTH_TOKEN = "AUTH_TOKEN" # Ensure to replace with your actual token
HEADERS = {'Authorization': f'Bearer {AUTH_TOKEN}', 'Content-Type': 'application/json'}
BASE_PATH = "."

## Function Definitions for Data Retrieval and Processing

This section contains a set of Python functions designed to interact with a monitoring system's API to retrieve and process metrics related to executor performance. These functions fetch time-series data, compute statistical summaries, and determine key temporal boundaries (start and end times).


In [ ]:
def fetch_metric_range(metric, executor_id, start_time, end_time, step='16s'):
    """Fetches a specific metric over a range of time for a given executor."""
    query = f'{metric}{{executor_id="{executor_id}"}}'
    params = {
        'query': query,
        'start': start_time,
        'end': end_time,
        'step': step
    }
    
    try:
        print(f"Querying range: {query} from {start_time} to {end_time} with step {step}")
        response = requests.get(f"{BASE_URL}/query_range", params=params, headers=HEADERS)
        response.raise_for_status()
        results = response.json().get('data', {}).get('result', [])
        
        if results:
            values = [float(item[1]) for item in results[0]['values']]
            return {
                'avg': np.mean(values),
                'min': np.min(values),
                'max': np.max(values),
                'sum': np.sum(values),
                'count': len(values),
                'stddev': np.std(values),
                'stdvar': np.var(values)
            }
        else:
            print(f"No data found for {metric} in range")
            return {agg: np.nan for agg in ['avg', 'min', 'max', 'sum', 'count', 'stddev', 'stdvar']}
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {metric} for {executor_id}: {e}")
        return {agg: np.nan for agg in ['avg', 'min', 'max', 'sum', 'count', 'stddev', 'stdvar']}


In [ ]:

def fetch_latest_end_time(executor_id, time_range="32d"):
    """Fetches the latest end time for a given executor_id within a time range."""
    query = f'worker_func_end_tstamp{{executor_id="{executor_id}"}}[{time_range}]'
    encoded_query = quote(query)
    url = f"{BASE_URL}/query?query={encoded_query}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        data = response.json()
        if data['status'] == 'success':
            if data['data']['result']:
                latest_timestamp = float('-inf')
                for result in data['data']['result']:
                    for timestamp, value in result['values']:
                        latest_timestamp = max(latest_timestamp, float(timestamp))
                return latest_timestamp
            else:
                print(f"No end time data found for executor_id={executor_id}")
                return None
        else:
            print(f"Error fetching end time: {data.get('error', 'Unknown error')}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching end time for {executor_id}: {e}")
        return None


def fetch_earliest_start_time(executor_id, time_range="32d"):
    """Fetches the earliest start time for a given executor_id within a time range."""
    query = f'worker_func_start_tstamp{{executor_id="{executor_id}"}}[{time_range}]'
    encoded_query = quote(query)
    url = f"{BASE_URL}/query?query={encoded_query}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        data = response.json()
        if data['status'] == 'success':
            if data['data']['result']:
                earliest_timestamp = float('inf')
                for result in data['data']['result']:
                    for timestamp, value in result['values']:
                        earliest_timestamp = min(earliest_timestamp, float(timestamp))
                return earliest_timestamp
            else:
                print(f"No start time data found for executor_id={executor_id}")
                return None
        else:
            print(f"Error fetching start time: {data.get('error', 'Unknown error')}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching start time for {executor_id}: {e}")
        return None


def fetch_start_end_times(executor_id):
    """Fetches the earliest start time and latest end time for an executor_id."""
    try:
        earliest_start_time = fetch_earliest_start_time(executor_id)
        latest_end_time = fetch_latest_end_time(executor_id)
        if earliest_start_time and latest_end_time:
            start_time_iso = datetime.utcfromtimestamp(earliest_start_time).isoformat()
            end_time_iso = datetime.utcfromtimestamp(latest_end_time).isoformat()
            return start_time_iso, end_time_iso
        else:
            return None, None
    except Exception as e:
        print(f"Error fetching start/end times for {executor_id}: {e}")
        return None, None

In [ ]:
def extract_executor_data(base_path):
    """Extracts executor data from files in the specified directory with the appropriate pattern."""
    executor_data = []
    pattern = re.compile(r'exec_ids_(\d+)_splits(?:_v\d+)?\.csv')
    
    for file_name in os.listdir(base_path):
        match = pattern.match(file_name)
        if match:
            file_path = os.path.join(base_path, file_name)
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                
                for _, row in df.iterrows():
                    executor_id = row['executor_id']
                    start_time, end_time = fetch_start_end_times(executor_id)
                    
                    if start_time and end_time:
                        try:
                            start_time_dt = datetime.fromisoformat(start_time)
                            end_time_dt = datetime.fromisoformat(end_time)
                            duration = (end_time_dt - start_time_dt).total_seconds()
                        except ValueError:
                            print(f"Error converting dates for executor_id {executor_id}.")
                            start_time_dt, end_time_dt, duration = None, None, None
                    else:
                        start_time_dt, end_time_dt, duration = None, None, None
                    
                    executor_data.append({
                        'executor_id': executor_id,
                        'total_price_usd': row['total_price_usd'],
                        'num_files': row['num_files'],
                        'splits': row['splits'],
                        'input_size_gb': row['input_size_gb'],
                        'runtime_memory_mb': row['runtime_memory_mb'],
                        'ephemeral_storage_mb': row['ephemeral_storage_mb'],
                        'worker_processes': row['worker_processes'],
                        'invoke_pool_threads': row['invoke_pool_threads'],
                        'vcpus': row['vcpus'],
                        'start_time': start_time_dt,
                        'end_time': end_time_dt,
                        'duration': duration
                    })
            else:
                print(f"File {file_path} does not exist.")
    
    df_executor_data = pd.DataFrame(executor_data)
    
    if df_executor_data.empty:
        print("Warning: No data was extracted, the resulting DataFrame is empty.")

    return df_executor_data

In [ ]:
# Extract data from executors
df_metrics_exd = extract_executor_data(BASE_PATH)
df_metrics_exd

## Data Processing: Feature Engineering and Outlier Removal

This section outlines two essential data processing functions used to enhance the dataset before applying machine learning models: adding derived features and removing outliers.

In [ ]:
def add_features(df):
    """Adds new features to the DataFrame to enhance the model."""
    df['memory_per_file'] = df['runtime_memory_mb'] / df['num_files']
    df['storage_per_file'] = df['ephemeral_storage_mb'] / df['num_files']
    df['vcpus_per_file'] = df['vcpus'] / df['num_files']
    df['files_per_vcpu'] = df['num_files'] / df['vcpus']
    df['size_per_file'] = df['input_size_gb'] / df['num_files']
    df['memory_per_gb'] = df['runtime_memory_mb'] / df['input_size_gb']
    df['vcpus_per_gb'] = df['vcpus'] / df['input_size_gb']
    df['storage_per_gb'] = df['ephemeral_storage_mb'] / df['input_size_gb']
    df['threads_per_worker'] = df['invoke_pool_threads'] / df['worker_processes']
    df['memory_per_thread'] = df['runtime_memory_mb'] / df['invoke_pool_threads']
    df['vcpus_per_thread'] = df['vcpus'] / df['invoke_pool_threads']
    df['memory_per_thread_vcpus_ratio'] = df['memory_per_thread'] / df['vcpus_per_thread']

    return df

In [ ]:
df_metrics = add_features(df_metrics_exd)
df_metrics

In [ ]:

data = df_metrics.drop(columns=['executor_id', 'Unnamed: 0', 'total_price_usd', 'start_time', 'end_time'])
data

In [ ]:
data.to_csv('dataset.csv', index=False)

## Data Preparation: Splitting

Preparing the data correctly is crucial for training a robust machine learning model. This step involves splitting the dataset into training and testing subsets and scaling the features to standardize their magnitudes.

### Splitting the Data into Training and Testing Sets
The dataset is divided into two subsets:
- **Training Set (70%)**: Used to train the model.
- **Testing Set (30%)**: Used to evaluate the model's performance on unseen data.

The splitting is performed using `train_test_split` from `sklearn.model_selection`. The `random_state=42` ensures reproducibility of the split.


In [ ]:
# Split data into 70% training and 30% testing
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

In [ ]:
# Separate features and labels
X_train = train_data.drop(columns=['duration'])
y_train = train_data['duration']

X_test = test_data.drop(columns=['duration'])
y_test = test_data['duration']

## Feature Scaling: Standardization

In this step, we standardize the features to ensure that all of them are on the same scale. Feature scaling is crucial when working with machine learning algorithms that are sensitive to the magnitude of features, such as gradient-based algorithms like XGBoost.

### **StandardScaler**
We use the `StandardScaler` from the `sklearn.preprocessing` module to standardize the features. This scaler transforms the data such that it has a mean of 0 and a standard deviation of 1, making the features comparable in terms of scale.


In [ ]:
# Scale the features
scaler = StandardScaler()

# Fit the scaler on the training set
X_train_scaled = scaler.fit_transform(X_train)

# Apply the scaler to the test set
X_test_scaled = scaler.transform(X_test)

## Data Augmentation: Adding Gaussian Noise

Data augmentation is a technique used to increase the diversity of the training dataset by artificially creating new samples. In this case, we augment the training data by adding Gaussian noise to the features. This helps the model generalize better by simulating slight variations in the data.

### Adding Gaussian Noise
Gaussian noise is generated with a mean of 0 and a specified standard deviation (`noise_level`). This noise is added to the features of the training dataset, creating a slightly modified version of the original data.


In [ ]:
def add_gaussian_noise(X, noise_level=0.01):
    """Adds Gaussian noise to the features."""
    noise = np.random.normal(loc=0.0, scale=noise_level, size=X.shape)
    return X + noise

In [ ]:
# Augment the training data with Gaussian noise
X_train_augmented = add_gaussian_noise(X_train_scaled, noise_level=0.02)

# Combine the original and augmented training data
X_train_final = np.vstack([X_train_scaled, X_train_augmented])
y_train_final = np.concatenate([y_train, y_train])

In [ ]:
print(f"Size of X_train_final: {X_train_final.shape}")
print(f"Size of y_train_final: {y_train_final.shape}")

## Hyperparameter Tuning with Optuna for XGBoost

This section details the process of optimizing hyperparameters for the XGBoost model using **Optuna**, a powerful and efficient hyperparameter tuning library. By leveraging advanced optimization techniques, the tuning process ensures that the model is tailored for optimal performance on the geospatial dataset.

### Overview of the Tuning Process

The optimization process involves defining a search space for the hyperparameters that govern the XGBoost model's behavior. Key parameters include the maximum depth of the trees, learning rate, number of estimators, subsampling ratios, regularization terms, and feature sampling ratios. These parameters influence the model's complexity, ability to generalize, and computational efficiency.

### Evaluation Strategy

The model is evaluated using cross-validation, which divides the training data into multiple folds to ensure robust evaluation. To enhance stability, the target variable is transformed logarithmically, and its distribution is stratified into quantile bins. This stratification ensures balanced splits, preventing skewed evaluations caused by uneven data distributions.

### Optuna Features

Optuna efficiently searches the hyperparameter space by combining random sampling with intelligent exploration:
- **Dynamic Pruning**: Trials that show poor early performance are terminated early, saving computational resources.
- **Parallel Trials**: Multiple hyperparameter combinations are evaluated concurrently, reducing tuning time.
- **Customized Objectives**: The optimization process directly targets minimizing the mean absolute error (MAE), aligning with the model's performance goals.

### Benefits of Optimization

The tuning process ensures the XGBoost model is highly effective for predicting the target variable by:
- Reducing overfitting through regularization and careful sampling.
- Balancing model complexity with computational efficiency.
- Leveraging cross-validation to enhance generalization capabilities.

This approach results in a fine-tuned XGBoost model capable of achieving optimal predictive performance for geospatial use cases, tailored to the nuances of the dataset and problem at hand.


In [ ]:
def objective_xgb(trial):
    """Objective function for XGBoost hyperparameter optimization using Optuna."""
    xgb_params = {
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.05),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 5000),
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'gamma': trial.suggest_uniform('gamma', 0.0, 0.5),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.01, 1.0),
        'eval_metric': 'mae',
        'objective': 'reg:squarederror',
        'verbosity': 0,
        'tree_method': 'hist'  # Use 'hist' to utilize CPU instead of GPU
    }

    xgb_model = XGBRegressor(**xgb_params, early_stopping_rounds=50, n_jobs=-1)

    # Training and validation with cross-validation
    X_train_array = np.array(X_train_final)
    y_train_array = np.array(y_train_final).flatten()

    y_train_log = np.log1p(y_train_array)
    
    num_bins = 5
    y_binned = pd.qcut(y_train_log, q=num_bins, labels=False, duplicates='drop')

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, valid_idx in cv.split(X_train_array, y_binned):
        X_t, X_v = X_train_array[train_idx], X_train_array[valid_idx]
        y_t, y_v = y_train_log[train_idx], y_train_log[valid_idx]

        xgb_model.fit(X_t, y_t, eval_set=[(X_v, y_v)], verbose=False)

        preds = xgb_model.predict(X_v)
        mae = mean_absolute_error(y_v, preds)
        scores.append(mae)

    return np.mean(scores)

In [ ]:
study_xgb = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
study_xgb.optimize(objective_xgb, n_trials=500, n_jobs=-1)

In [ ]:
best_params_xgb = study_xgb.best_params
print(f"Best hyperparameters: {best_params_xgb}")

In [ ]:
xgb_best = XGBRegressor(**best_params_xgb, early_stopping_rounds=50, eval_metric='rmse')

## Final Model Training and Predictions

### Training the XGBoost Model

With the optimal hyperparameters obtained through Optuna, the final XGBoost model is trained on the augmented training dataset. The target variable is log-transformed using `np.log1p()` to manage its skewed distribution and ensure that the model learns effectively from the data. 

During training:
- The test set, with its log-transformed target, is used as the validation set for evaluation.
- XGBoost monitors the validation set performance to adjust its learning process dynamically.

### Generating Predictions

Once the model is trained, predictions are made on the scaled test dataset:
1. Predictions are initially made in the log-transformed space to match the training target's transformation.
2. These predictions are reverted to their original scale using the exponential transformation `np.expm1()` for meaningful interpretation.

### Key Outcomes

This step finalizes the model's ability to generalize its learning to unseen data, ensuring accurate predictions while retaining interpretability. The use of logarithmic transformation and inverse scaling ensures the handling of non-linearities and data skewness effectively.


In [ ]:
# Train the model with the final training data
y_train_log = np.log1p(y_train_final)
xgb_best.fit(X_train_final, y_train_log, eval_set=[(X_test_scaled, np.log1p(y_test))], verbose=True)

In [ ]:
# Predict and transform the predictions back to the original scale
y_pred_log = xgb_best.predict(X_test_scaled)
y_pred = np.expm1(y_pred_log)

## Model Evaluation and Analysis

### Metrics Calculation
To evaluate the performance of the trained XGBoost model, the following metrics are computed:
1. **Mean Absolute Error (MAE):** Measures the average absolute differences between actual and predicted values, providing insight into overall prediction accuracy.
2. **Mean Absolute Percentage Error (MAPE):** Reflects prediction accuracy as a percentage of the actual values, offering an intuitive understanding of the error magnitude.
3. **R² Score:** Indicates the proportion of variance in the dependent variable that the model explains, demonstrating the quality of fit.

### Cross-Validation
The model undergoes 10-fold cross-validation to ensure its robustness. During this process:
- The dataset is divided into 10 subsets, with 9 used for training and 1 for validation iteratively.
- The mean MAE from all folds is calculated, providing a reliable measure of the model's performance across varying data partitions.

### Configuration Analysis
The test dataset is analyzed to identify:
1. The configuration yielding the **lowest actual duration**.
2. The configuration with the **lowest predicted duration**.
   
This comparison helps validate whether the model correctly identifies the optimal configuration.

### Relative Error Analysis
Relative errors are calculated as the absolute differences between actual and predicted durations, normalized by the actual values. A histogram visualizes the distribution of relative errors in the test set:
- A tighter distribution indicates higher predictive accuracy.
- Outliers, if present, suggest cases where the model struggles to generalize.

### Visualization
The distribution of relative errors is plotted to assess the model's reliability in predictions across different samples. 

### Insights
1. If the configurations with the lowest actual and predicted durations match, the model demonstrates excellent predictive alignment.
2. A low MAPE indicates the model's consistency in making precise predictions relative to the scale of the target variable.

### Summary
This evaluation not only confirms the model's predictive capabilities but also highlights areas for further refinement by analyzing instances of higher errors or mismatched optimal configurations.


In [ ]:
# Calculate MAE, MAPE, and R² on the original scale
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE: {mape:.2%}")

r2 = r2_score(y_test, y_pred)
print(f"R²: {r2:.4f}")

In [ ]:
xgb_best_cv = XGBRegressor(**best_params_xgb)

# Perform cross-validation and calculate the mean MAE
cv_scores = cross_val_score(xgb_best_cv, X_train_final, y_train_final, cv=10, scoring='neg_mean_absolute_error')
print(f"Average MAE in cross-validation: {-cv_scores.mean():.2f}")

In [ ]:
# Create a DataFrame with the test features and add the actual and predicted durations
df_test = pd.DataFrame(X_test, columns=[
    'splits', 'num_files', 'input_size_gb', 'runtime_memory_mb', 'ephemeral_storage_mb', 
    'worker_processes', 'invoke_pool_threads', 'vcpus', 'memory_per_file', 'storage_per_file', 
    'vcpus_per_file', 'files_per_vcpu', 'size_per_file', 'memory_per_gb', 'vcpus_per_gb', 
    'storage_per_gb', 'threads_per_worker', 'memory_per_thread', 'vcpus_per_thread'
])
df_test['duration'] = y_test
df_test['predicted_duration'] = y_pred

# Identify the configuration with the lowest actual duration and the lowest predicted duration
min_duration_row = df_test.loc[df_test['duration'].idxmin()]
min_predicted_duration_row = df_test.loc[df_test['predicted_duration'].idxmin()]

# Print both configurations
print("\nConfiguration with the lowest actual duration:")
print(min_duration_row)

print("\nConfiguration with the lowest predicted duration:")
print(min_predicted_duration_row)

# Check if they are the same row
if min_duration_row.equals(min_predicted_duration_row):
    print("\nThe configuration with the lowest actual duration is the same as the one with the lowest predicted duration.")
else:
    print("\nThe configuration with the lowest actual duration is NOT the same as the one with the lowest predicted duration.")

In [ ]:
# Calculate relative errors
relative_errors = np.abs(y_test - y_pred) / y_test

# Visualize the distribution of relative errors
plt.figure(figsize=(10, 6))
sns.histplot(relative_errors, bins=30, kde=True, color='purple')
plt.xlabel('Relative Error')
plt.ylabel('Frequency')
plt.title('Distribution of Relative Errors in the Test Set')
plt.show()

In [ ]:
# Calculate the MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE in the test set: {mape:.2%}")

## Residual and Scatter Plot Analysis

### Residual Plot
The residual plot visualizes the differences between actual and predicted durations, providing insights into the model's errors:
- **Residuals:** Computed as the actual values minus the predicted values, they indicate whether predictions systematically overestimate or underestimate the actual durations.
- **Key Features:**
  - A horizontal red dashed line at zero residual indicates perfect predictions.
  - Ideally, residuals should scatter randomly around this line without clear patterns, suggesting that the model captures the data trends effectively.
  - Systematic patterns in residuals may indicate underfitting, overfitting, or unaccounted-for complexities in the data.

### Scatter Plot of Actual vs. Predicted Durations
This plot compares actual and predicted durations to assess the model's accuracy visually:
- **Key Features:**
  - A red dashed line represents the ideal fit where predictions perfectly match actual values.
  - Data points closer to the line indicate more accurate predictions.
  - Points deviating significantly suggest instances where the model struggles to make accurate predictions.

### Purpose
These visualizations provide an intuitive understanding of:
1. The accuracy of the predictions across the dataset.
2. Potential biases or inconsistencies in the model.
3. Specific areas where model improvements may be necessary.

### Insights
- **Residual Plot:** Helps identify heteroscedasticity (variance inconsistency) or patterns indicating model issues.
- **Scatter Plot:** Highlights the model's ability to predict durations effectively, with closer alignment to the ideal fit suggesting higher predictive performance.


In [ ]:
# Plot residuals
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_pred, y=residuals, alpha=0.6, color='g')
plt.axhline(0, color='r', linestyle='--')
plt.xlabel('Predicted Duration')
plt.ylabel('Residuals (Actual - Predicted)')
plt.title('Residual Plot')
plt.show()

In [ ]:
# Scatter plot of actual vs predicted durations
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6, color='b')
plt.plot(
    [y_test.min(), y_test.max()],
    [y_test.min(), y_test.max()],
    color='r', linestyle='--', label='Ideal Fit'
)
plt.xlabel('Actual Duration')
plt.ylabel('Predicted Duration')
plt.title('Actual vs Predicted Duration')
plt.legend()
plt.show()

## Baseline Model and Learning Curve Analysis

### Baseline Prediction
The baseline prediction uses the mean value of the training data as a simple model for comparison:
- **Baseline MAE:** This metric measures the mean absolute error when predicting the same mean value for all test samples.
- **Improvement Over Baseline:** Calculated as the percentage reduction in MAE achieved by the model compared to the baseline.

#### Insights:
- A significant improvement over the baseline MAE indicates that the model has learned patterns beyond a simple average prediction.

### Learning Curve Analysis
The learning curve helps visualize how the model's performance changes as the training set size increases:
- **Training Errors:** Reflect how well the model fits the training data.
- **Testing Errors:** Indicate generalization ability on unseen data.

#### Key Features:
- **Early Stopping:** Prevents overfitting by halting training if performance on a validation set does not improve after a set number of rounds.
- **Trend Analysis:** 
  - A decreasing training error with increasing data suggests improved model fitting.
  - A small gap between training and testing errors implies good generalization.
  - A persistent gap might indicate overfitting or underfitting.

### Histogram of Actual vs Predicted Durations
This histogram compares the distributions of actual and predicted durations:
- **Kernel Density Estimation (KDE):** Provides a smoothed visualization of the underlying distributions.
- **Key Observations:**
  - A high overlap between the two curves suggests accurate predictions.
  - Divergences indicate areas where the model may struggle to capture complexities in the data.

#### Overall Importance
These analyses provide a comprehensive understanding of:
1. The model's relative performance against a naive baseline.
2. Its learning dynamics and generalization capability.
3. The alignment of predicted distributions with actual outcomes.


In [ ]:
# Baseline prediction using mean of training data
y_baseline = np.full_like(y_test, y_train.mean())
mae_baseline = mean_absolute_error(y_test, y_baseline)
print(f"MAE of the baseline model: {mae_baseline:.2f}")

# Calculate improvement
improvement_mae = 100 * (mae_baseline - mae) / mae_baseline
print(f"Improvement over baseline MAE: {improvement_mae:.2f}%")

In [ ]:
def plot_learning_curve(model, X_train, y_train, X_test, y_test, early_stopping_rounds=50):
    """Plots the learning curve for a given model using MAE as the metric."""
    train_errors, test_errors = []
    kf = KFold(n_splits=10, shuffle=True, random_state=42)  # Use KFold for regression
    
    for m in range(1, len(X_train)):
        # Ensure X_train[:m] and y_train[:m] have the same dimensions
        X_train_subset = X_train[:m]
        y_train_subset = y_train[:m]
        
        # Train the model using a validation set (eval_set) for early stopping
        model.fit(
            X_train_subset, y_train_subset, 
            eval_set=[(X_test, y_test)], 
            verbose=False
        )
        
        # Calculate errors on the training and test sets using MAE
        y_train_predict = model.predict(X_train_subset)
        y_test_predict = model.predict(X_test)
        
        train_errors.append(mean_absolute_error(y_train_subset, y_train_predict))  # MAE on training set
        test_errors.append(mean_absolute_error(y_test, y_test_predict))  # MAE on test set
    
    # Plot the learning curves with MAE
    plt.plot(train_errors, "r-+", linewidth=2, label="train (MAE)")
    plt.plot(test_errors, "b-", linewidth=3, label="test (MAE)")
    plt.xlabel("Training set size")
    plt.ylabel("MAE")
    plt.legend()
    
    # Save the figure in SVG format
    plt.savefig("learning_curve.svg", format="svg")
    plt.show()

In [ ]:
# Call the function to plot the learning curve with early stopping rounds
plot_learning_curve(xgb_best, X_train_final, y_train_final, X_test_scaled, y_test, early_stopping_rounds=50)

In [ ]:
# Create the histogram with KDE
plt.figure(figsize=(10, 6))
sns.histplot(y_test, color='blue', label='Actual', kde=True)
sns.histplot(y_pred, color='red', label='Predicted', kde=True)

# Add labels and title
plt.title('Histogram of Actual and Predicted Values with KDE')
plt.xlabel('Duration')
plt.ylabel('Frequency')
plt.legend()

# Show the plot
plt.show()

## Linear Regression Model: Analysis and Comparison with XGBoost

### Linear Regression Model
1. **Model Training:** A simple linear regression model is trained using the scaled features from the training dataset. This approach assumes a linear relationship between the input features and the target variable (duration).
2. **Performance Metrics:**
   - **Mean Absolute Error (MAE):** Quantifies the average absolute difference between actual and predicted values.
   - **Mean Absolute Percentage Error (MAPE):** Indicates prediction error as a percentage of actual values, useful for relative comparisons.
   - **R² Score:** Measures the proportion of variance in the target variable explained by the model. Higher values close to 1 indicate better performance.

### Comparison with XGBoost
1. **Performance Metrics for XGBoost:**
   - Lower MAE and MAPE values demonstrate XGBoost's ability to model complex, non-linear relationships better than linear regression.
   - A higher R² score signifies a better fit and predictive power.
2. **Improvement Over Linear Regression:**
   - The percentage improvement in MAE highlights the superiority of XGBoost in handling the given data complexity.

### Prediction Comparison Visualization
- **Scatter Plot:**
  - Compares actual durations against predictions from both models.
  - Points closer to the red diagonal (ideal fit) indicate more accurate predictions.
  - XGBoost predictions show better alignment with actual durations compared to linear regression, as evidenced by reduced scatter.

### Key Takeaways
1. **Model Suitability:** While linear regression provides a baseline understanding, its simplicity limits its performance on complex datasets.
2. **XGBoost Advantages:** Leveraging advanced tree-based methods, XGBoost captures intricate patterns and outperforms linear regression on all evaluated metrics.
3. **Practical Implications:** For datasets with non-linear relationships and feature interactions, XGBoost or other advanced models are preferred over linear regression.


In [ ]:
# Create the linear regression model
linear_model = LinearRegression()

# Fit the model to the scaled training data
linear_model.fit(X_train_final, y_train_final)

In [ ]:
# Predict durations with the scaled test set
y_pred_linear = linear_model.predict(X_test_scaled)

# Calculate MAE, MAPE, and R²
mae_linear = mean_absolute_error(y_test, y_pred_linear)
mape_linear = mean_absolute_percentage_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

print(f"Linear Regression - MAE: {mae_linear:.2f}")
print(f"Linear Regression - MAPE: {mape_linear:.2%}")
print(f"Linear Regression - R²: {r2_linear:.4f}")

In [ ]:
print("Comparison of Models:")
print(f"XGBoost - MAE: {mae:.2f}, MAPE: {mape:.2%}, R²: {r2:.4f}")
print(f"Linear Regression - MAE: {mae_linear:.2f}, MAPE: {mape_linear:.2%}, R²: {r2_linear:.4f}")

# Compare the improvement in MAE between XGBoost and linear regression
improvement_over_linear = 100 * (mae_linear - mae) / mae_linear
print(f"Improvement over Linear Regression MAE: {improvement_over_linear:.2f}%")

In [ ]:
# Plot prediction comparisons
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6, color='blue', label='XGBoost Predictions')
sns.scatterplot(x=y_test, y=y_pred_linear, alpha=0.6, color='green', label='Linear Regression Predictions')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', label='Ideal Fit')
plt.xlabel('Actual Duration')
plt.ylabel('Predicted Duration')
plt.title('Actual vs Predicted Duration: XGBoost vs Linear Regression')
plt.legend()
plt.show()

## PCA + Linear Regression: Model Analysis and Comparison

### Principal Component Analysis (PCA) for Dimensionality Reduction
1. **Goal of PCA:** PCA is used to reduce the dimensionality of the data by transforming it into a set of linearly uncorrelated variables (principal components). This transformation helps in improving the model's performance by focusing on the most significant features and reducing noise from less important dimensions.
2. **Choosing the Number of Components:** In this case, `n_components` is set to 6, meaning the model uses the first 6 principal components. This number can be adjusted depending on the data variance captured by these components.
3. **Training the Model:**
   - PCA is applied to both the training and test datasets.
   - A linear regression model is then trained on the transformed training data (using the principal components), and predictions are made on the test set.

### Performance Metrics for PCA + Linear Regression
1. **MAE (Mean Absolute Error):** A lower MAE indicates that the predictions are closer to the actual values, suggesting the model's accuracy.
2. **MAPE (Mean Absolute Percentage Error):** A useful percentage metric for assessing relative prediction accuracy.
3. **R² Score:** Measures how well the model explains the variance in the data. Higher values indicate better predictive performance.

### Model Comparison: XGBoost vs Linear Regression vs PCA + Linear Regression
- **XGBoost:** A tree-based ensemble method that is well-suited for capturing complex relationships in the data. It generally performs better than linear models in terms of handling non-linearities and feature interactions.
- **Linear Regression:** A basic linear approach that is easy to interpret but often less effective for complex datasets.
- **PCA + Linear Regression:** By using PCA to reduce dimensionality, we simplify the dataset before applying linear regression. While this may help in cases with high multicollinearity or noise, it may still not capture complex relationships as well as XGBoost.

### MAE Improvement Comparison
1. **Improvement Over Linear Regression:** XGBoost significantly outperforms linear regression, as expected, due to its ability to capture non-linear relationships.
2. **Improvement Over PCA + Linear Regression:** PCA improves the linear regression model by reducing dimensionality and potentially mitigating overfitting, but it still does not match the performance of XGBoost.

### Prediction Comparison Visualization
- **Scatter Plot:** The plot compares the actual vs predicted values for each model:
  - XGBoost predictions align better with the actual values.
  - Linear Regression and PCA + Linear Regression also make predictions, but they show more deviation from the ideal fit (red dashed line).

### Key Takeaways
1. **XGBoost is the best model** for this dataset, especially for capturing complex patterns and interactions.
2. **PCA + Linear Regression** can be a useful alternative when dealing with highly-dimensional data but does not match the predictive power of more complex models like XGBoost.
3. **Linear Regression** serves as a baseline model, helpful for understanding the simplest possible relationships in the data but often outperformed by more advanced methods.

### Conclusion
In summary, while linear regression and PCA + linear regression are useful techniques for baseline and reduced-dimensionality models, XGBoost stands out in terms of predictive accuracy. The use of PCA with linear regression provides some improvement but still falls short compared to XGBoost's capabilities in handling non-linear relationships and complex feature interactions.


In [ ]:
# Define the number of principal components for PCA
n_components = 6  # Adjust this number as needed

# Apply PCA on the training set
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_final)
X_test_pca = pca.transform(X_test_scaled)

# Create the linear regression model
linear_model_pca = LinearRegression()

# Fit the model to the PCA-reduced training data
linear_model_pca.fit(X_train_pca, y_train_final)

# Predict durations on the PCA-reduced test set
y_pred_pca = linear_model_pca.predict(X_test_pca)

# Calculate MAE, MAPE, and R² for linear regression with PCA
mae_pca = mean_absolute_error(y_test, y_pred_pca)
mape_pca = mean_absolute_percentage_error(y_test, y_pred_pca)
r2_pca = r2_score(y_test, y_pred_pca)

print(f"PCA + Linear Regression - MAE: {mae_pca:.2f}")
print(f"PCA + Linear Regression - MAPE: {mape_pca:.2%}")
print(f"PCA + Linear Regression - R²: {r2_pca:.4f}")

In [ ]:
print("\nComparison of Models:")
print(f"XGBoost - MAE: {mae:.2f}, MAPE: {mape:.2%}, R²: {r2:.4f}")
print(f"Linear Regression - MAE: {mae_linear:.2f}, MAPE: {mape_linear:.2%}, R²: {r2_linear:.4f}")
print(f"PCA + Linear Regression - MAE: {mae_pca:.2f}, MAPE: {mape_pca:.2%}, R²: {r2_pca:.4f}")

# Compare the improvement in MAE between XGBoost and the linear regression models
improvement_over_linear = 100 * (mae_linear - mae) / mae_linear
improvement_over_pca_linear = 100 * (mae_pca - mae) / mae_pca

print(f"Improvement over Linear Regression MAE: {improvement_over_linear:.2f}%")
print(f"Improvement over PCA + Linear Regression MAE: {improvement_over_pca_linear:.2f}%")

In [ ]:
# Plot prediction comparisons
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6, color='blue', label='XGBoost Predictions')
sns.scatterplot(x=y_test, y=y_pred_linear, alpha=0.6, color='green', label='Linear Regression Predictions')
sns.scatterplot(x=y_test, y=y_pred_pca, alpha=0.6, color='orange', label='PCA + Linear Regression Predictions')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', label='Ideal Fit')
plt.xlabel('Actual Duration')
plt.ylabel('Predicted Duration')
plt.title('Actual vs Predicted Duration: XGBoost vs Linear Regression vs PCA + Linear Regression')
plt.legend()

# Save the plot as an SVG file
plt.savefig("act_vs_pred_duration.svg", format='svg')

# Show the plot
plt.show()

### Cross-Validation and Residual Analysis

- **Linear Regression (Cross-Validation MAE):**
  We performed cross-validation on the Linear Regression model using 5 folds and calculated the **Mean Absolute Error (MAE)** for each fold. The average MAE was computed and provided as the model's performance metric.
  
- **PCA + Linear Regression (Cross-Validation MAE):**
  Similarly, we applied Principal Component Analysis (PCA) to reduce the dimensionality of the data before performing Linear Regression. Cross-validation was also conducted on this PCA-transformed data, and the average MAE was computed.

- **Comparison of Models:**
  The **average MAE** values from the cross-validation results of both models are as follows:
  - **Linear Regression**: MAE of `average_mae_cv_linear`.
  - **PCA + Linear Regression**: MAE of `average_mae_cv_pca`.

- **Residuals Analysis:**
  The residuals for the following models were calculated by subtracting the predicted values from the actual values:
  - **XGBoost residuals**: The difference between the predicted values from XGBoost and the actual test data.
  - **Linear Regression residuals**: The difference between the predicted values from the Linear Regression model and the actual test data.
  - **PCA + Linear Regression residuals**: The difference between the PCA + Linear Regression model's predicted values and the actual test data.

- **Comparative Residual Plot:**
  A residual plot was created comparing the residuals for XGBoost, Linear Regression, and PCA + Linear Regression. This plot helps visually assess the spread of residuals for each model, with the following features:
  - **XGBoost residuals** are plotted in green.
  - **Linear Regression residuals** are plotted in blue.
  - **PCA + Linear Regression residuals** are plotted in red.
  
  A horizontal line at 0 is added to indicate no residual error.

- **Saved Output:**
  The comparative residual plot was saved as an SVG file titled `residual_plot_comparison.svg`.


In [ ]:
# Create a Linear Regression model
linear_model_cv = LinearRegression()

# Define MAE as a scorer for cross-validation
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Perform cross-validation for Linear Regression
mae_cv_linear = cross_val_score(linear_model_cv, X_train_final, y_train_final, 
                                scoring=mae_scorer, cv=5)

In [ ]:
# Create a Linear Regression model for PCA-transformed data
linear_model_pca_cv = LinearRegression()

# Perform cross-validation for PCA + Linear Regression
mae_cv_pca = cross_val_score(linear_model_pca_cv, X_train_pca, y_train_final, 
                             scoring=mae_scorer, cv=5)

# Calculate the average MAE from the cross-validation results and convert to positive
average_mae_cv_linear = -np.mean(mae_cv_linear)
average_mae_cv_pca = -np.mean(mae_cv_pca)

In [ ]:
print(f"Average MAE (Cross-Validation) - Linear Regression: {average_mae_cv_linear:.2f}")
print(f"Average MAE (Cross-Validation) - PCA + Linear Regression: {average_mae_cv_pca:.2f}")

In [ ]:
# Calculate residuals for each model
residuals_xgb = y_test - y_pred
residuals_linear = y_test - y_pred_linear
residuals_pca = y_test - y_pred_pca


In [ ]:
# Plot comparative residuals
plt.figure(figsize=(15, 8))

# XGBoost residuals
sns.scatterplot(x=y_pred, y=residuals_xgb, alpha=0.6, color='g', label='XGBoost Residuals')

# Linear regression residuals
sns.scatterplot(x=y_pred_linear, y=residuals_linear, alpha=0.6, color='b', label='Linear Regression Residuals')

# PCA + Linear regression residuals
sns.scatterplot(x=y_pred_pca, y=residuals_pca, alpha=0.6, color='r', label='PCA + Linear Regression Residuals')

plt.axhline(0, color='k', linestyle='--')
plt.xlabel('Predicted Duration')
plt.ylabel('Residuals (Actual - Predicted)')
plt.title('Residual Plot Comparison: XGBoost vs. Linear Regression vs. PCA + Linear Regression')
plt.legend()

# Save the plot as an SVG file
plt.savefig("residual_plot_comparison.svg", format='svg')
plt.show()